In [57]:
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as                           EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException, WebDriverException, NoSuchElementException, ElementClickInterceptedException

from bs4 import BeautifulSoup
import random
import time
import re

### Some variables

In [2]:
FIREFOX_DRIVER = "/home/nick/apps/geckodriver"
PROBLEMS_FILE = "linkuri_probleme.txt"
FREE_PROBLEMS_FILE = "linkuri_free_probleme.txt"

In [3]:
website_url = "https://leetcode.com/problemset/"
service = FirefoxService(FIREFOX_DRIVER)  # Exemplu: /usr/local/bin/geckodriver
fremium_problems = []

### Driver options:

In [4]:
def dump_page(bs: BeautifulSoup, filename: str="file.html") -> None:
    with open(filename, "w") as f1:
        f1.write(bs.prettify())

In [5]:
def dump_fremium_problems():
    global fremium_problems
    with open(FREE_PROBLEMS_FILE, "r") as g:
        fremium_problems = g.readlines()
def dump_all_problems():
    global problemset_links
    with open(PROBLEMS_FILE, "r") as g:
        problemset_links = g.readlines()

## SKIP to finding Discussion

In [52]:
# dump_fremium_problems()
dump_all_problems()
# print(len(fremium_problems))
# print(fremium_problems)

# Webdriver's code:

In [50]:
firefox_options = Options()
firefox_options.add_argument("--headless")

# Inițializează driverul pentru Firefox
driver = webdriver.Firefox(service=service, options=firefox_options)

In [51]:
# Accesează pagina dorită
driver.get(website_url)
driver.set_page_load_timeout(3)
print("Am accesat pagina cu Firefox")

Am accesat pagina cu Firefox


#### Sari 2 celule daca nu vrei sa recitesti toate problemele

In [40]:
f = open(PROBLEMS_FILE, "a")

# Asta se va asigura ca fisierul exista si este gol
# with open(PROBLEMS_FILE, 'w'):
#     pass

Modific codul asta doar dupa ce voi strange mai mult de o pagina de probleme

In [43]:
wait = WebDriverWait(driver, 10)
problemset_links = []
# Iterează prin numarul de paginii de pe leetcode practice
i = 1

for problems_page in range(70):
    try:
        anchors = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.TAG_NAME, "a")))
        anchors = [a for a in anchors if re.match(r"^\d+\.", a.text)] # Aici am pastrat doar ancorele cu problemset-uri efectiv
        
        # Itereaza prin linkurile de pe pagina respectiva
        for anchor in anchors:
            try:
                link_text = anchor.text
                link_url = anchor.get_attribute("href")
            
                problemset_links.append(link_url)
                f.write(f"{link_url}\n")
            except TimeoutException as e:
                print(f"TimeoutException la pagina {i}")
                time.sleep(random.randint(3, 7))
    except StaleElementReferenceException as e:
        print(f"StaleElementReferenceException la pagina {i}")
        continue

    try:
        next_page = wait.until(
            EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='next']"))
        )
        next_page.click()
        wait.until(EC.staleness_of(anchors[0]))
    except TimeoutException as e:
        print(f"TimeoutException la pagina {i} -> cel mai probabil {i} este ultima pagina")
        break
    i += 1

StaleElementReferenceException la pagina 2
TimeoutException la pagina 67 -> cel mai probabil 67 este ultima pagina


TEST

In [53]:
print(problemset_links[156])
print(len(problemset_links))

https://leetcode.com/problems/binary-tree-upside-down

3315


#### Mai sari 3 celule sa ajungi la 'Discussions' topic
# Extract fremium problemset
#### Prima mare iteratie va fi printre problemele free

In [58]:
var = 0
linkuri_problematice = []
def test_is_premium(link: str) -> bool:
    global var, driver
    var += 1
    if var % 40 == 0:
        print(f"i = {var}")
    try:
        driver.get(link)
        subscrieburi = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.TAG_NAME, "a")))
        premium = False
        for sub in subscrieburi:
            if re.match(r"^Subscribe", sub.text):
                premium = True
                break
        driver.back()
        # print(f'i = {var} bine')
        return premium
    except StaleElementReferenceException:
        linkuri_problematice.append(link)
        print(f"i = {var} -> \'StaleElementReferenceException\' la linkul {link}")
        return True
    except TimeoutException:
        linkuri_problematice.append(link)
        print(f"i = {var} -> TimeoutException la linkul {link}")
        time.sleep(random.randint(3, 7)) # Asteptam impotriva bannarii
        return True
    except WebDriverException as ee:
        if "out of memory" in str(ee):
            print(f"i = {var} 'out of memory' occured. Restarting driver")
            driver.quit()
            driver = webdriver.Firefox(service=service, options=firefox_options)
            return test_is_premium(link)
        else:
            return True
            

In [59]:
fremium_problems = [link for link in problemset_links if not test_is_premium(link)]
# with open(FREE_PROBLEMS_FILE, "w") as file:
#     for fremium in fremium_problems:
#         file.write(fremium + '\n')

i = 34 -> TimeoutException la linkul https://leetcode.com/problems/search-in-rotated-sorted-array

i = 35 -> TimeoutException la linkul https://leetcode.com/problems/find-first-and-last-position-of-element-in-sorted-array

i = 40
i = 56 -> TimeoutException la linkul https://leetcode.com/problems/jump-game

i = 68 -> 'StaleElementReferenceException' la linkul https://leetcode.com/problems/add-binary

i = 79 -> TimeoutException la linkul https://leetcode.com/problems/subsets

i = 80
i = 80 -> TimeoutException la linkul https://leetcode.com/problems/word-search

i = 81 -> TimeoutException la linkul https://leetcode.com/problems/remove-duplicates-from-sorted-array-ii

i = 82 -> TimeoutException la linkul https://leetcode.com/problems/search-in-rotated-sorted-array-ii

i = 83 -> TimeoutException la linkul https://leetcode.com/problems/remove-duplicates-from-sorted-list-ii

i = 84 -> TimeoutException la linkul https://leetcode.com/problems/remove-duplicates-from-sorted-list

i = 100 -> Timeo

In [47]:
# Numele fisierului tau
nume_fisier = "linkuri_free_probleme.txt"

# Citim toate liniile din fisier, eliminam cele goale si rescriem fisierul
with open(nume_fisier, "r") as f:
    linii = f.readlines()

# Filtram liniile goale
linii_fara_goluri = [linie for linie in linii if linie.strip()]

# Rescriem fisierul fara liniile goale
with open(nume_fisier, "w") as f:
    f.writelines(linii_fara_goluri)

print("Fisierul a fost actualizat fara liniile libere.")


TEST

In [61]:
print(len(fremium_problems))

2449


TEST

In [43]:
driver.get(problemset_links[0])

### finding 'Discussion' section

In [69]:
import random
idx = random.randint(0, len(fremium_problems))
driver.get(fremium_problems[idx])

In [139]:
link_de_test = "https://leetcode.com/problems/remove-duplicates-from-sorted-list-ii/description/"
driver.get(link_de_test)

In [140]:
try:
    discussion_element = driver.find_element(By.XPATH, "//*[contains(text(), 'Discussion')]")
    discussion_element.click()  # Dă click pe element
    print("Click pe elementul 'Discussion' efectuat cu succes!")
except Exception as e:
    print(f"Eroare: {e}")

Click pe elementul 'Discussion' efectuat cu succes!


TEXT

In [144]:
def go_to_next_comment_page(drv, timeout=1):
    try:
        # drv.find_element(By.XPATH, "//button[@aria-label='next']")
        next_button = WebDriverWait(drv, timeout).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='next']"))
            )
        next_button.click()
        return True
    except (NoSuchElementException, ElementClickInterceptedException, TimeoutException):
        return False

def expand_read_more(parent_div) -> bool:
    try:
        read_more_button = WebDriverWait(parent_div, 1).until(
            EC.element_to_be_clickable((By.XPATH, ".//div[contains(@class, 'text-md flex w-full items-center justify-center text-label-1 dark:text-dark-label-1')]"))
        )
        read_more_button.click()
        expand_read_more(parent_div)
        return True
    except (NoSuchElementException, ElementClickInterceptedException, TimeoutException):
        return False

def has_responses(div) -> bool:
    try:
        responses = WebDriverWait(div, 2).until(
            EC.element_to_be_clickable((By.XPATH, ".//div[contains(@class, 'flex items-center gap-1 group shrink-0 cursor-pointer transition-colors')]"))
        )
        return True
    except (NoSuchElementException, TimeoutException):
        return False


daca nu am sarit de user_links:

In [145]:
user_links = []

In [146]:
pag = 1
global_idx = 0
user_timeout = 2.2
DISCUSSION_PAGE = "comentarii_discutii_pagina.txt"
with open(DISCUSSION_PAGE, "w") as g:
    pass


with open(DISCUSSION_PAGE, "w") as f:
    while True:
        try:
            f.write(f"PAGINA {pag}\n" + "=" * 100 + "\n")
            user_anchors = WebDriverWait(driver, user_timeout).until(
                EC.presence_of_all_elements_located((By.XPATH, "//a[starts-with(@href, '/u/') and normalize-space(text()) != '']"))
            )
            # afiseaza userii de pe acelasi nivel
            for idx, user_anchor in enumerate(user_anchors):
                try:
                    sixth_parent_div = user_anchor.find_element(By.XPATH, "ancestor::div[6]")
                    if expand_read_more(sixth_parent_div):
                        print(f"page {pag}  -> comment {idx + global_idx + 1} had a 'read more' button")
                    else:
                        print(f"page {pag} -> com {idx + global_idx + 1}")
                    
                    if has_responses(sixth_parent_div):
                        print("\tHAS RESPONSES!")
                    
                    second_child_div = sixth_parent_div.find_element(By.XPATH, "./div[2]")
                    fourth_descendant_div = second_child_div.find_element(By.XPATH, "./descendant::div[4]")
                    comment_texts = fourth_descendant_div.find_elements(By.XPATH, ".//p")
            
                    f.write(f"{idx + global_idx + 1} . user: {user_anchor.text}  | URL: {user_anchor.get_attribute('href')}\n")
                    for comment_idx, comment_text in enumerate(comment_texts):
                        f.write(f"comment {comment_idx + 1} : {comment_text.text}\n" + "=" * 30 + "\n")
                except StaleElementReferenceException:
                    # print(f"Encountered a stale element on page {pag} at element {idx+1}, retrying on this user...")
                    break
                    
            else:
                global_idx += len(user_anchors)
                if go_to_next_comment_page(driver):
                    pag += 1
                else:
                    break
                    
        except StaleElementReferenceException:
            print(f"Page {pag} elements went stale, retrying on this page...")
            continue

page 1 -> com 1
	HAS RESPONSES!
page 1 -> com 2
	HAS RESPONSES!
page 1 -> com 3
	HAS RESPONSES!
page 1 -> com 4
	HAS RESPONSES!
page 1 -> com 5
page 1 -> com 6
page 1 -> com 7
	HAS RESPONSES!
page 1 -> com 8
	HAS RESPONSES!
page 1 -> com 9
	HAS RESPONSES!
page 1 -> com 10
	HAS RESPONSES!
page 2 -> com 11
page 2  -> comment 11 had a 'read more' button
	HAS RESPONSES!
page 2 -> com 12
page 2 -> com 13
page 2 -> com 14
page 2 -> com 15
	HAS RESPONSES!
page 2 -> com 16
page 2 -> com 17
	HAS RESPONSES!
page 2 -> com 18
	HAS RESPONSES!
page 2 -> com 19
page 2 -> com 20
page 3 -> com 21
page 3 -> com 22
page 3 -> com 23
page 3 -> com 24
page 3 -> com 25
page 3 -> com 26
page 3 -> com 27
page 3 -> com 28
page 3 -> com 29
page 3 -> com 30
page 4 -> com 31
page 4 -> com 31
page 4 -> com 32
page 4 -> com 33
page 4 -> com 34
page 4 -> com 35
page 4 -> com 36
page 4 -> com 37
page 4 -> com 38
page 4  -> comment 39 had a 'read more' button
page 4 -> com 40
page 5 -> com 41
page 5 -> com 42
page 5 ->

'read_more' css class: <br>
text-md flex w-full items-center justify-center text-label-1 dark:text-dark-label-1 <br>
text-md flex w-full items-center justify-center text-label-1 dark:text-dark-label-1

Buton de comentarii: <br>
viewBox="0 0 24 24"

TEST

In [218]:
driver.get(user_links[0][1])

Ma duc la o eticheta de tip 'div' cu 6 nivele mai sus de eticheta cu numele userului, si ulterior tot 6 nivele in jos incepand cu al doilea div

In [33]:
driver.quit()

In [36]:
driver.get(problemset_links[156])

TODO: sa gasesc butonul de 'read more', sa il apas si sa iau tot continutul.
TODO: sa apas pe raspunsuri la comment de fiecare data recursiv pana nu mai exista

|- c1
|  | - c11
|  | - c12
|- c2
|  | - c21
|  | - c22
|  | - c23
|  
|- c3
|- c4